In [188]:
import warnings
from decimal import Decimal
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='tSkWbLL1f6F3L_AXGVEgyoQpdPZk53bCnADpJicQl9lm',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ckd-donotdelete-pr-ra1bwm7tptvduy'
object_key = 'ckd_dataset.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
# ignore warnings generated due to usage of old version of tensorflow
warnings.simplefilter("ignore")

In [189]:
df

,bp,al,bu,sg,sod,hemo,pcv,rc,htn,dm,classification
0,80,1,36.0,1.020,138.0,15.4,44,5.2,1,1,1
1,50,4,18.0,1.020,138.0,11.3,38,4.8,0,0,1
2,80,2,53.0,1.010,138.0,9.6,31,4.8,0,1,1
3,70,4,56.0,1.005,111.0,11.2,32,3.9,1,0,1
4,80,2,26.0,1.010,138.0,11.6,35,4.6,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
395,80,0,49.0,1.020,150.0,15.7,47,4.9,0,0,0
396,70,0,31.0,1.025,141.0,16.5,54,6.2,0,0,0
397,80,0,26.0,1.020,137.0,15.8,49,5.4,0,0,0
398,60,0,50.0,1.025,135.0,14.2,51,5.9,0,0,0


In [190]:
# Separating the dependent and independent variables
y = df['classification']
X = df.drop(['classification'], axis = 1)
X.head()

,bp,al,bu,sg,sod,hemo,pcv,rc,htn,dm
0,80,1,36.0,1.020,138.0,15.4,44,5.2,1,1
1,50,4,18.0,1.020,138.0,11.3,38,4.8,0,0
2,80,2,53.0,1.010,138.0,9.6,31,4.8,0,1
3,70,4,56.0,1.005,111.0,11.2,32,3.9,1,0
4,80,2,26.0,1.010,138.0,11.6,35,4.6,0,0


Feature Selection using Extra Tree Classifier

In [192]:
X.head()

,bp,al,bu,sg,sod,hemo,pcv,rc,htn,dm
0,80,1,36.0,1.020,138.0,15.4,44,5.2,1,1
1,50,4,18.0,1.020,138.0,11.3,38,4.8,0,0
2,80,2,53.0,1.010,138.0,9.6,31,4.8,0,1
3,70,4,56.0,1.005,111.0,11.2,32,3.9,1,0
4,80,2,26.0,1.010,138.0,11.6,35,4.6,0,0


In [193]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = None)

**Gradient Boosting Classifier**

In [194]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

# accuracy score, confusion matrix and classification report of gradient boosting classifier
gb_acc = accuracy_score(y_test, gb.predict(X_test))

print(f"Training Accuracy of Gradient Boosting Classifier is {accuracy_score(y_train, gb.predict(X_train))}")
print(f"Test Accuracy of Gradient Boosting Classifier is {gb_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, gb.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, gb.predict(X_test))}")

Training Accuracy of Gradient Boosting Classifier is 1.0
Test Accuracy of Gradient Boosting Classifier is 100.0 

Confusion Matrix :- 
[[46  0]
 [ 0 74]]

Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
           1       1.00      1.00      1.00        74

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



**Stochastic Gradient Boosting**

In [195]:
sgb = GradientBoostingClassifier(max_depth = 4, subsample = 0.90, max_features = 0.75, n_estimators = 200)
sgb.fit(X_train, y_train)

# accuracy score, confusion matrix and classification report of stochastic gradient boosting classifier

sgb_acc = accuracy_score(y_test, sgb.predict(X_test))

print(f"Training Accuracy of Stochastic Gradient Boosting is {accuracy_score(y_train, sgb.predict(X_train))*100}")
print(f"Test Accuracy of Stochastic Gradient Boosting is {sgb_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, sgb.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, sgb.predict(X_test))}")

Training Accuracy of Stochastic Gradient Boosting is 100.0
Test Accuracy of Stochastic Gradient Boosting is 100.0 

Confusion Matrix :- 
[[46  0]
 [ 0 74]]

Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
           1       1.00      1.00      1.00        74

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



**Extra Tree Classifier**

In [196]:
# accuracy score, confusion matrix and classification report of extra tree classifier
etc = ExtraTreesClassifier()
etc.fit(X_train, y_train)

etc_acc = accuracy_score(y_test, etc.predict(X_test))

print(f"Training Accuracy of Extra Trees Classifier is {accuracy_score(y_train, etc.predict(X_train))*100}")
print(f"Test Accuracy of Extra Trees Classifier is {etc_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, etc.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, etc.predict(X_test))}")

Training Accuracy of Extra Trees Classifier is 100.0
Test Accuracy of Extra Trees Classifier is 100.0 

Confusion Matrix :- 
[[46  0]
 [ 0 74]]

Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
           1       1.00      1.00      1.00        74

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



**KNN**

In [197]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# accuracy score, confusion matrix and classification report of knn
knn_acc = accuracy_score(y_test, knn.predict(X_test))
print(f"Training Accuracy of KNN is {accuracy_score(y_train, knn.predict(X_train))*100}")
print(f"Test Accuracy of KNN is {knn_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, knn.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, knn.predict(X_test))}")

Training Accuracy of KNN is 94.28571428571428
Test Accuracy of KNN is 91.66666666666666 

Confusion Matrix :- 
[[45  1]
 [ 9 65]]

Classification Report :- 
               precision    recall  f1-score   support

           0       0.83      0.98      0.90        46
           1       0.98      0.88      0.93        74

    accuracy                           0.92       120
   macro avg       0.91      0.93      0.91       120
weighted avg       0.93      0.92      0.92       120



**Decision Tree Classifier**

In [198]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

# accuracy score, confusion matrix and classification report of decision tree

dtc_acc = accuracy_score(y_test, dtc.predict(X_test))

print(f"Training Accuracy of Decision Tree Classifier is {accuracy_score(y_train, dtc.predict(X_train))*100}")
print(f"Test Accuracy of Decision Tree Classifier is {dtc_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, dtc.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, dtc.predict(X_test))}")


Training Accuracy of Decision Tree Classifier is 100.0
Test Accuracy of Decision Tree Classifier is 97.5 

Confusion Matrix :- 
[[46  0]
 [ 3 71]]

Classification Report :- 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97        46
           1       1.00      0.96      0.98        74

    accuracy                           0.97       120
   macro avg       0.97      0.98      0.97       120
weighted avg       0.98      0.97      0.98       120



In [199]:
# hyper parameter tuning of decision tree 

from sklearn.model_selection import GridSearchCV
grid_param = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [3, 5, 7, 10],
    'splitter' : ['best', 'random'],
    'min_samples_leaf' : [1, 2, 3, 5, 7],
    'min_samples_split' : [1, 2, 3, 5, 7],
    'max_features' : ['auto', 'sqrt', 'log2']
}

grid_search_dtc = GridSearchCV(dtc, grid_param, cv = 5, n_jobs = -1, verbose = 1)
grid_search_dtc.fit(X_train, y_train)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 7, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 5, 7],
                         'min_samples_split': [1, 2, 3, 5, 7],
                         'splitter': ['best', 'random']},
             verbose=1)

In [200]:
# best estimator
dtc = grid_search_dtc.best_estimator_

# accuracy score, confusion matrix and classification report of decision tree
dtc_acc = accuracy_score(y_test, dtc.predict(X_test))

print(f"Training Accuracy of Decision Tree Classifier is {accuracy_score(y_train, dtc.predict(X_train))*100}")
print(f"Test Accuracy of Decision Tree Classifier is {dtc_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, dtc.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, dtc.predict(X_test))}")

Training Accuracy of Decision Tree Classifier is 97.85714285714285
Test Accuracy of Decision Tree Classifier is 99.16666666666667 

Confusion Matrix :- 
[[46  0]
 [ 1 73]]

Classification Report :- 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        46
           1       1.00      0.99      0.99        74

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120



**Random Forest Classifier**

In [201]:
from sklearn.ensemble import RandomForestClassifier

rd_clf = RandomForestClassifier(criterion = 'entropy', max_depth = 11, max_features = 'auto', min_samples_leaf = 2, min_samples_split = 3, n_estimators = 130)
rd_clf.fit(X_train, y_train)

# accuracy score, confusion matrix and classification report of random forest

rd_clf_acc = accuracy_score(y_test, rd_clf.predict(X_test))

print(f"Training Accuracy of Random Forest Classifier is {accuracy_score(y_train, rd_clf.predict(X_train))*100}")
print(f"Test Accuracy of Random Forest Classifier is {rd_clf_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, rd_clf.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, rd_clf.predict(X_test))}")

Training Accuracy of Random Forest Classifier is 99.28571428571429
Test Accuracy of Random Forest Classifier is 100.0 

Confusion Matrix :- 
[[46  0]
 [ 0 74]]

Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
           1       1.00      1.00      1.00        74

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



**ADA Boost Classifier**

In [202]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(base_estimator = dtc)
ada.fit(X_train, y_train)

# accuracy score, confusion matrix and classification report of ada boost

ada_acc = accuracy_score(y_test, ada.predict(X_test))

print(f"Training Accuracy of Ada Boost Classifier is {accuracy_score(y_train, ada.predict(X_train))*100}")
print(f"Test Accuracy of Ada Boost Classifier is {ada_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, ada.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, ada.predict(X_test))}")

Training Accuracy of Ada Boost Classifier is 100.0
Test Accuracy of Ada Boost Classifier is 100.0 

Confusion Matrix :- 
[[46  0]
 [ 0 74]]

Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
           1       1.00      1.00      1.00        74

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



**Cat Boost Classifier**

In [203]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [204]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(iterations=10)
cat.fit(X_train, y_train)

Learning rate set to 0.408198
0:	learn: 0.4614342	total: 660us	remaining: 5.95ms
1:	learn: 0.3601524	total: 1.13ms	remaining: 4.51ms
2:	learn: 0.2803005	total: 1.61ms	remaining: 3.75ms
3:	learn: 0.2342306	total: 2.05ms	remaining: 3.07ms
4:	learn: 0.1970805	total: 2.5ms	remaining: 2.5ms
5:	learn: 0.1694721	total: 2.95ms	remaining: 1.97ms
6:	learn: 0.1389511	total: 3.24ms	remaining: 1.39ms
7:	learn: 0.1185999	total: 3.56ms	remaining: 890us
8:	learn: 0.1066939	total: 4.04ms	remaining: 449us
9:	learn: 0.0961551	total: 4.5ms	remaining: 0us


In [205]:
cat_acc = accuracy_score(y_test, cat.predict(X_test))

print(f"Training Accuracy of Cat Boost Classifier is {accuracy_score(y_train, cat.predict(X_train))*100}")
print(f"Test Accuracy of Cat Boost Classifier is {cat_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, cat.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, cat.predict(X_test))}")

Training Accuracy of Cat Boost Classifier is 99.28571428571429
Test Accuracy of Cat Boost Classifier is 100.0 

Confusion Matrix :- 
[[46  0]
 [ 0 74]]

Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
           1       1.00      1.00      1.00        74

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



**Logistic Regression Classifier**

In [206]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(random_state=0).fit(X_train, y_train)

In [207]:
log_acc = accuracy_score(y_test, log.predict(X_test))

print(f"Training Accuracy of Logistic Regression Classifier is {accuracy_score(y_train, log.predict(X_train))*100}")
print(f"Test Accuracy of Logistic Regression Classifier is {log_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, log.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, log.predict(X_test))}")

Training Accuracy of Logistic Regression Classifier is 94.64285714285714
Test Accuracy of Logistic Regression Classifier is 97.5 

Confusion Matrix :- 
[[46  0]
 [ 3 71]]

Classification Report :- 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97        46
           1       1.00      0.96      0.98        74

    accuracy                           0.97       120
   macro avg       0.97      0.98      0.97       120
weighted avg       0.98      0.97      0.98       120



**Linear Discriminant Analysis Classifier**

In [208]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [209]:
lda_acc = accuracy_score(y_test, log.predict(X_test))

print(f"Training Accuracy of Linear Discriminant Analysis Classifier is {accuracy_score(y_train, lda.predict(X_train))*100}")
print(f"Test Accuracy of Linear Discriminant Analysis Classifier is {lda_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, lda.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, lda.predict(X_test))}")

Training Accuracy of Linear Discriminant Analysis Classifier is 96.42857142857143
Test Accuracy of Linear Discriminant Analysis Classifier is 97.5 

Confusion Matrix :- 
[[46  0]
 [ 4 70]]

Classification Report :- 
               precision    recall  f1-score   support

           0       0.92      1.00      0.96        46
           1       1.00      0.95      0.97        74

    accuracy                           0.97       120
   macro avg       0.96      0.97      0.97       120
weighted avg       0.97      0.97      0.97       120



**GaussianNB**

In [210]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [211]:
gnb_acc = accuracy_score(y_test, log.predict(X_test))

print(f"Training Accuracy of GaussianNB Classifier is {accuracy_score(y_train, gnb.predict(X_train))*100}")
print(f"Test Accuracy of GaussianNB Classifier is {gnb_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, gnb.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, gnb.predict(X_test))}")

Training Accuracy of GaussianNB Classifier is 95.0
Test Accuracy of GaussianNB Classifier is 97.5 

Confusion Matrix :- 
[[46  0]
 [11 63]]

Classification Report :- 
               precision    recall  f1-score   support

           0       0.81      1.00      0.89        46
           1       1.00      0.85      0.92        74

    accuracy                           0.91       120
   macro avg       0.90      0.93      0.91       120
weighted avg       0.93      0.91      0.91       120



**Support Vector Machine Classifier**

In [212]:
from sklearn import svm
svmc=svm.SVC()
svmc.fit(X_train, y_train)

SVC()

In [213]:
svmc_acc = accuracy_score(y_test, log.predict(X_test))

print(f"Training Accuracy of Support Vector Machine Classifier is {accuracy_score(y_train, svmc.predict(X_train))*100}")
print(f"Test Accuracy of Support Vector Machine Classifier is {svmc_acc*100} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, svmc.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, svmc.predict(X_test))}")

Training Accuracy of Support Vector Machine Classifier is 86.07142857142858
Test Accuracy of Support Vector Machine Classifier is 97.5 

Confusion Matrix :- 
[[45  1]
 [16 58]]

Classification Report :- 
               precision    recall  f1-score   support

           0       0.74      0.98      0.84        46
           1       0.98      0.78      0.87        74

    accuracy                           0.86       120
   macro avg       0.86      0.88      0.86       120
weighted avg       0.89      0.86      0.86       120



# Models Comparison

In [214]:
models = pd.DataFrame({
    'Model' : [ 'KNN', 'Decision Tree Classifier', 'Random Forest Classifier','ADA Boost Classifier',
             'Gradient Boosting Classifier', 'Stochastic Gradient Boosting', 'Cat Boost', 'Extra Tree Classifier', 
               'Logistic Regression Classifier', 'Linear Discriminant Analysis', 'GaussianNB', 'Support Vector Machine Classifier'],
    'Score' : [knn_acc, dtc_acc, rd_clf_acc, ada_acc, gb_acc, sgb_acc, cat_acc, etc_acc, log_acc, lda_acc, gnb_acc, svmc_acc]
})

models.sort_values(by = 'Score', ascending = False)

,Model,Score
2,Random Forest Classifier,1.000000
3,ADA Boost Classifier,1.000000
4,Gradient Boosting Classifier,1.000000
5,Stochastic Gradient Boosting,1.000000
6,Cat Boost,1.000000
7,Extra Tree Classifier,1.000000
1,Decision Tree Classifier,0.991667
8,Logistic Regression Classifier,0.975000
9,Linear Discriminant Analysis,0.975000
10,GaussianNB,0.975000


In [215]:
import plotly.express as px
px.bar(data_frame = models, x = 'Score', y = 'Model', color = 'Score', template = 'ggplot2', 
       title = 'Models Comparison')

In [216]:
from sklearn.ensemble import VotingClassifier
vot_reg = VotingClassifier(estimators=[('adaboost', ada), ('gradientboost', gb), ('extratree', etc)], voting='hard')

In [217]:
vot_reg.fit(X_train, y_train)

VotingClassifier(estimators=[('adaboost',
                              AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10,
                                                                                       max_features='sqrt',
                                                                                       min_samples_split=7,
                                                                                       splitter='random'))),
                             ('gradientboost', GradientBoostingClassifier()),
                             ('extratree', ExtraTreesClassifier())])

In [218]:
y_train_predict=vot_reg.predict(X_train)
y_test_predict=vot_reg.predict(X_test)

In [219]:
from sklearn import metrics
print("-------Test Data--------")
print('MAE:', metrics.mean_absolute_error(y_test, y_test_predict))
print('MSE:', metrics.mean_squared_error(y_test, y_test_predict))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_test_predict)))

print("\n-------Train Data--------")
print('MAE:', metrics.mean_absolute_error(y_train,y_train_predict))
print('MSE:', metrics.mean_squared_error(y_train, y_train_predict))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_train_predict)))

-------Test Data--------
MAE: 0.0
MSE: 0.0
RMSE: 0.0

-------Train Data--------
MAE: 0.0
MSE: 0.0
RMSE: 0.0


In [220]:
import pickle
pickle.dump(vot_reg, open("ckd_ensemble_model.pkl", "wb"))

In [221]:
pwd

'/home/wsuser/work'

In [222]:
!pip install -U ibm-watson-machine-learning

In [223]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np

In [224]:
wml_credentials = {
"apikey":"nsbW84O5-58I1rSeLI_8YhWJ7ZmphElqVB5ZMf1g1i8S", 
"url":"https://us-south.ml.cloud.ibm.com"
}

In [225]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ----  ------------------------
ID                                    NAME  CREATED
5bbc03d9-eb65-47a7-94c2-dfc9cb98c13d  CKD   2022-11-17T18:33:00.787Z
------------------------------------  ----  ------------------------


In [226]:
SPACE_ID = "5bbc03d9-eb65-47a7-94c2-dfc9cb98c13d"

In [227]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [228]:
wml_client.software_specifications.list(500)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [229]:
import sklearn
sklearn.__version__

'1.0.2'

In [230]:
MODEL_NAME = 'CKD'
DEPLOYMENT_NAME = 'CKD_deployname'
DEMO_MODEL = vot_reg

In [231]:
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [232]:
model_props = {
wml_client.repository.ModelMetaNames.NAME: 'CKD',
wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

In [233]:
model_details = wml_client.repository.store_model(
model=DEMO_MODEL,
meta_props=model_props,
training_data=X_train,
training_target=y_train
)

In [234]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'classification',
  'schemas': {'input': [{'fields': [{'name': 'bp', 'type': 'int64'},
      {'name': 'al', 'type': 'int64'},
      {'name': 'bu', 'type': 'float64'},
      {'name': 'sg', 'type': 'float64'},
      {'name': 'sod', 'type': 'float64'},
      {'name': 'hemo', 'type': 'float64'},
      {'name': 'pcv', 'type': 'int64'},
      {'name': 'rc', 'type': 'float64'},
      {'name': 'htn', 'type': 'int64'},
      {'name': 'dm', 'type': 'int64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-17T19:19:28.182Z',
  'id': '5d9f3b22-170c-4e71-b565-475fc1973a5d',
  'modified_at': '2022-11-17T19:19:31.588Z',
  'name': 'CKD',
  'owner': 'IBMid-661003Y8VM',
  'resource_key': '15e8322a-65b6-438c-bd04-4a700c96a485',
  'space_id': '5bbc03d9-eb65-47a7

In [235]:
model_id=wml_client.repository.get_model_id(model_details)

In [236]:
model_id

'5d9f3b22-170c-4e71-b565-475fc1973a5d'

In [237]:
deployment_props = {
wml_client.deployments.ConfigurationMetaNames.NAME:'CKD',
wml_client.deployments.ConfigurationMetaNames.ONLINE:{}
}

In [238]:
deployment = wml_client.deployments.create(
artifact_uid=model_id,
meta_props=deployment_props
)



#######################################################################################

Synchronous deployment creation for uid: '5d9f3b22-170c-4e71-b565-475fc1973a5d' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='f1dfeb85-c1e4-4b8e-a479-a4364b22c6fd'
------------------------------------------------------------------------------------------------


